In [37]:
import sys
sys.path.append(
    "/Users/shenshen/drake-build/install/lib/python3.7/site-packages")

import pydrake
import numpy as np
from numpy.linalg import eig, inv
# import pydrake.symbolic as sym
from pydrake.all import (MathematicalProgram, Polynomial, SolutionResult,
                         Solve, Jacobian, Evaluate, RealContinuousLyapunovEquation, Substitute,Variables, Expression, MosekSolver)
from scipy.linalg import solve_lyapunov, solve_discrete_lyapunov

Some helper functions, can skip

In [38]:
#  Some helper functions
class opt(object):

    def __init__(self, nX, degL1=4, degL2=4, degV=2, explicit_add = False, init_with_free = False):
        self.degV = degV
        self.max_iterations = 10
        self.converged_tol = .01
        self.degL1 = degL1
        self.degL2 = degL2
        self.nX = nX
        self.explicit_add = explicit_add
        self.init_with_free = init_with_free
        if init_with_free:
            assert explicit_add 
        

def balanceQuadForm(S, P):
    # copied from the old drake, with only syntax swap
    #  Quadratic Form "Balancing"
    #
    #    T = balqf(S,P)
    #
    #  Input:
    #    S -- n-by-n symmetric positive definite.
    #    P -- n-by-n symmetric, full rank.
    #
    #  Finds a T such that:
    #    T'*S*T = D
    #    T'*P*T = D^(-1)

    # if np.linalg.norm(S - S.T, 1) > 1e-8:
        # raise Error('S must be symmetric')
    # if np.linalg.norm(P - P.T, 1) > 1e-8:
        # raise Error('P must be symmetric')
    # if np.linalg.cond(P) > 1e10:
        # raise Error('P must be full rank')

    # Tests if S positive def. for us.
    V = np.linalg.inv(np.linalg.cholesky(S).T)
    [N, l, U] = np.linalg.svd((V.T.dot(P)).dot(V))
    if N.ravel()[0] < 0:
        N = -N
    T = (V.dot(N)).dot(np.diag(np.power(l, -.25, dtype=float)))
    D = np.diag(np.power(l, -.5, dtype=float))
    return T, D

def balance(x, V, f, S, A):
    if S is None:
        S = .5 * (Substitute(Jacobian(Jacobian(V, x).T, x), x, 0 * x))
    if A is None:
        J = Jacobian(f, x)
        env = dict(zip(x, np.zeros(x.shape)))
        mapping = dict(zip(x, x0))
        A = np.array([[i.Evaluate(env) for i in j]for j in J])

    [T, D] = balanceQuadForm(S, (S@A + A.T@S))
    # print('T is %s' % (T))
    # Sbal = (T.T)@(S)@(T)
    Vbal = V.Substitute(dict(zip(x, T@x)))
    # print([i.Substitute(dict(zip(x,T@x))) for i in f])
    fbal = inv(T)@[i.Substitute(dict(zip(x, T@x))) for i in f]
    return T, Vbal, fbal, S, A

def clean(poly, tol=1e-9):
    if isinstance(poly, Expression):
        poly = Polynomial(poly)
    return poly.RemoveTermsWithSmallCoefficients(tol).ToExpression()

Fix at just one iteration. We'll only output Mosek verbose info for `optimizeV` function:

In [43]:
def bilinear(x, V0, f, S0, A, options):
    V = V0
    [T, V0bal, fbal, S0, A] = balance(x, V0, f, S0, A)
    rho = 1
    vol = 0
    for iter in range(1):
        last_vol = vol
        # balance on every iteration (since V and Vdot are changing):
        [T, Vbal, fbal] = balance(x, V, f, S0 / rho, A)[0:3]
        V0bal = V0.Substitute(dict(zip(x, T@x)))
        [x, L1, sigma1] = findL1(x, fbal, Vbal, options)
        [x, L2] = findL2(x, Vbal, V0bal, rho, options)
        [x, Vbal, rho] = optimizeV(x, fbal, L1, L2, V0bal, sigma1, options)
        vol = rho

        #  undo balancing (for the next iteration, or if i'm done)
        V = Vbal.Substitute(dict(zip(x, inv(T)@x)))
        if ((vol - last_vol) < options.converged_tol * last_vol):
            break
    return V


def findL1(old_x, f, V, options):
    print('finding L1')
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(options.nX, 'l1x')
    V = V.Substitute(dict(zip(list(V.GetVariables()), x)))
    f = np.array([i.Substitute(dict(zip(list(i.GetVariables()), x))) for i in f])
    # % construct multipliers for Vdot
    if options.init_with_free:
        L1 = prog.NewFreePolynomial(Variables(x), options.degL1).ToExpression()
    else:
        L1 = prog.NewSosPolynomial(Variables(x), options.degL1)[0].ToExpression()
    # % construct Vdot
    Vdot = clean(V.Jacobian(x) @ f)
    # % construct slack var
    sigma1 = prog.NewContinuousVariables(1, "s")[0]
    prog.AddConstraint(sigma1 >= 0)
    # % setup SOS constraints
    prog.AddSosConstraint(-Vdot + L1 * (V - 1) - sigma1 * V)
    if (options.explicit_add):
        prog.AddSosConstraint(L1)
    # add cost
    prog.AddCost(-sigma1)
    # result = Solve(prog)
    solver = MosekSolver()
    solver.set_stream_logging(False, "")
    result = solver.Solve(prog, None, None)
#     print(result.get_solution_result())
    assert result.is_success()
    L1 = (result.GetSolution(L1))
    sigma1 = result.GetSolution(sigma1)
    print('sigma1 is %s' % (sigma1))
    return x, L1, sigma1

def findL2(old_x, V, V0, rho, options):
    print('finding L2')
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(options.nX, "l2x")
    V = (V.Substitute(dict(zip(list(V.GetVariables()), x))))
    V0 = (V0.Substitute(dict(zip(list(V0.GetVariables()), x))))
        # % construct multipliers for Vdot
    if options.init_with_free:
        L2 = prog.NewFreePolynomial(Variables(x), options.degL2).ToExpression()
    else:
        L2 = prog.NewSosPolynomial(Variables(x), options.degL2)[0].ToExpression()
    # % construct slack var
    slack = prog.NewContinuousVariables(1, "s")[0]
    prog.AddConstraint(slack >= 0)
    # add normalizing constraint
    prog.AddSosConstraint(-(V - 1) + L2 * (V0 - rho))
    if (options.explicit_add):
        prog.AddSosConstraint(L2)
    prog.AddCost(slack)
    solver = MosekSolver()
    solver.set_stream_logging(False, "")
    result = solver.Solve(prog, None, None)
#     print(result.get_solution_result())
    assert result.is_success()
    L2 = (result.GetSolution(L2))
    return x, L2

def optimizeV(old_x, f, L1, L2, V0, sigma1, options):
    print('finding V')
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(options.nX, "Vx")
    L1 = (L1.Substitute(dict(zip(list(L1.GetVariables()), x))))
    L2 = (L2.Substitute(dict(zip(list(L2.GetVariables()), x))))
    V0 = (V0.Substitute(dict(zip(list(V0.GetVariables()), x))))
    f = np.array([i.Substitute(dict(zip(list(i.GetVariables()), x))) for i in f])
    
    #% construct V
    if options.init_with_free:
        V = prog.NewFreePolynomial(Variables(x), options.degV).ToExpression()
    else:
        V = prog.NewSosPolynomial(Variables(x), options.degV)[0].ToExpression()
        
    Vdot = V.Jacobian(x) @ f
    # % construct rho
    rho = prog.NewContinuousVariables(1, "r")[0]
    prog.AddConstraint(rho >= 0)

    # % setup SOS constraints
    prog.AddSosConstraint(-Vdot + L1 * (V - 1) - sigma1 * V / 2)
    prog.AddSosConstraint(-(V - 1) + L2 * (V0 - rho))
    if (options.explicit_add):
        prog.AddSosConstraint(V)
            
    # % run SeDuMi/MOSEK and check output
    prog.AddCost(-rho)
    solver = MosekSolver()
    solver.set_stream_logging(True, "")
    solver.Solve(prog, None, None)
    result = Solve(prog)
    print(result.get_solution_result())
    assert result.is_success()
    V = result.GetSolution(V)
    rho = result.GetSolution(rho)
#     print('rho is %s' % (rho))
    return x, V, rho

Setup the plant information and some initialization

In [44]:

plant = 'vdp'
prog = MathematicalProgram()
if plant is 'Cubic':
    nx = 1
    x = prog.NewIndeterminates(nx, "x")
    xdot = [-x[0] + x[0]**3]
elif plant is 'vdp':
    nx=2
    x = prog.NewIndeterminates(nx, "x")
    xdot = -np.array([x[1], -x[0] - x[1] * (x[0]**2 - 1)])

J = Jacobian(xdot, x)
env = dict(zip(x, np.zeros(x.shape)))
A = np.array([[i.Evaluate(env) for i in j]for j in J])
S0 = solve_lyapunov(A.T, -np.eye(nx))
V0 = (x.T@S0@x)

## To show discrepency 

We use `explicit_add` to control the optimization problem setup of `optimizeV`. Depending on the boolean value of `explicit_add`, we add (or not add) a SOS constraint on $V$ (whetehr or not it's declared `Free` or `Sos` at construction)

We also use `init_with_free` to control the optimization problem setup of `optimizeV`. Depending on the boolean value of `init_with_free`, we construct $V$ to be either `Free` or `Sos`



In [47]:
# if we initialize the decision variables as free polynomials and add 
init_with_free = True
explicit_add = True
options = opt(nx, degL1=6, degL2=6, degV=4, explicit_add = explicit_add,init_with_free=init_with_free)
bilinear(x, V0, xdot, S0, A, options)

finding L1
sigma1 is 0.31711568522649486
finding L2
finding V
SolutionResult.kSolutionFound


<Expression "(-1.3652889643846632e-09 - 4.9046795022147686e-19 * (0.4793254849758784 * Vx(0) + 0.77556492636499841 * Vx(1)) + 6.7632728665017964e-18 * (0.9865338258102101 * Vx(0) - 0.60971143540217843 * Vx(1)) + 0.024331377297312409 * ((0.4793254849758784 * Vx(0) + 0.77556492636499841 * Vx(1)) * (0.9865338258102101 * Vx(0) - 0.60971143540217843 * Vx(1))) + 3.4453553715109266e-17 * ((0.4793254849758784 * Vx(0) + 0.77556492636499841 * Vx(1)) * pow((0.9865338258102101 * Vx(0) - 0.60971143540217843 * Vx(1)), 2)) - 0.093308893752344149 * ((0.4793254849758784 * Vx(0) + 0.77556492636499841 * Vx(1)) * pow((0.9865338258102101 * Vx(0) - 0.60971143540217843 * Vx(1)), 3)) - 1.4894294840412125e-16 * (pow((0.4793254849758784 * Vx(0) + 0.77556492636499841 * Vx(1)), 2) * (0.9865338258102101 * Vx(0) - 0.60971143540217843 * Vx(1))) + 0.085111834897226235 * (pow((0.4793254849758784 * Vx(0) + 0.77556492636499841 * Vx(1)), 2) * pow((0.9865338258102101 * Vx(0) - 0.60971143540217843 * Vx(1)), 2)) + 0.1425221

In [48]:
# if we initialize the decision variables as sos polynomials but still add SOS constraint later
init_with_free = False
explicit_add = True
options = opt(nx, degL1=6, degL2=6, degV=4, explicit_add = explicit_add,init_with_free=init_with_free)
bilinear(x, V0, xdot, S0, A, options)

finding L1
sigma1 is 0.3171156841233344
finding L2
finding V
SolutionResult.kUnknownError


AssertionError: 

Here's the matlab printout of 1st iteration `optimizeV` call:

![title](matlabprintout.png)


In [7]:
# Print out of not adding explict SOS constraint
explicit_add = True
options = opt(nx, degL1=6, degL2=6, degV=4, explicit_add = explicit_add)
bilinear(x, V0, xdot, S0, A, options)

finding L1
sigma1 is 0.3171156841233344
finding L2
finding V
SolutionResult.kUnknownError


AssertionError: 